Just make some demos

In [ ]:
from poppy.creatures import PoppyTorso
import numpy as np
import time

poppy = PoppyTorso(simulator='vrep')

In [ ]:
poppy.motors

In [2]:
poppy.reset_simulation()
io = poppy._controllers[0].io
name = 'cube'
position = [0, -0.15, 0.8] # X, Y, Z
sizes = [0.01, 0.01, 0.01] # in meters
mass = 0 # in kg
io.add_cube(name, position, sizes, mass)



In [5]:
# draw circle

io.get_object_position('cube')
io.set_object_position('cube', position=[0, -0.1, 0])
while 1:
    t = time.time()
    while time.time() - t < 0.0001:
        1
    x = np.sin(2 * 3.14159 * time.time() * 0.1) * 0.5
    z = np.cos(2 * 3.14159 * time.time() * 0.1) * 0.5
    io.set_object_position('cube', position=[x, -2, z + 1])

NameError: name 'np' is not defined

In [18]:
# draw single line
t = time.time()

while 1:
    x = np.sin(2 * 3.14159 * time.time() * 0.1)
    z = np.cos(2 * 3.14159 * time.time() * 0.1) * 0.5
    io.set_object_position('cube', position=[x, -1, 1])

    if time.time() - t > 0.1:
        # print canSeeJudge(headForwardDirection(), objectRelPosition(name))
        t = time.time()
        test = canSeeJudge(headForwardDirection(), objectRelPosition(name))

        test1, test2 = getState(test, 5,5)
        if not test1:
            print test1

29.8420059357
27.4671505489
24.5989510135
21.4910462618
18.5932165653
15.231466425
11.6688061384
8.12598361612
4.53174279924
1.10454248022
2.55319977883
7.22877069907
10.7143803634
14.0329267746
17.4610245655
20.768761737
23.8316686464
26.5518144642
28.6721023728
31.3818975506
33.5770770822
35.4496686738
37.2995139304
The object can't be seen
False
38.8527569063
The object can't be seen
False
40.1268759129
The object can't be seen
False
41.4620079066
The object can't be seen
False
42.5623601393
The object can't be seen
False
43.4493506006
The object can't be seen
False
44.2889368628
The object can't be seen
False
44.9752253386
The object can't be seen
False
45.48968217
The object can't be seen
False


KeyboardInterrupt: 

In [13]:
poppy.head_z.goal_position = 0

In [23]:
poppy.head_y.goal_position = 0

In [ ]:
poppy.head_z.present_position

In [ ]:
print math.sin(25.3 / 180 * 3.14159)

In [19]:
def headForwardDirection():
    angleNegativeY = poppy.head_z.present_position
    angleSurfaceXY = - poppy.head_y.present_position

    angleNegativeY = angleNegativeY / 180 * 3.14159
    angleSurfaceXY = angleSurfaceXY / 180 * 3.14159

    y = - np.cos(angleSurfaceXY) * np.cos(angleNegativeY)
    x = np.cos(angleSurfaceXY) * np.sin(angleNegativeY)
    z = np.sin(angleSurfaceXY)

    forwardDire = [x, y, z]
    return forwardDire
def objectRelPosition(name):
    objectPos = io.get_object_position(name)
    positionCameraOri = [0, -0.05, 1.06]

    objectRelPos = [objectPos[i] - positionCameraOri[i] for i in xrange(3)]

    return objectRelPos
def canSeeJudge(headForwardDirection, objectRelPos):
    orthognalBasis1 = headForwardDirection
    orthognalBasis2 = [orthognalBasis1[1], -orthognalBasis1[0], 0]
    normOrthBasis2 = np.linalg.norm(orthognalBasis2)
    orthognalBasis2 =  [orthognalBasis2[i] / normOrthBasis2 for i in xrange(3) ]
    orthognalBasis3 = np.cross(orthognalBasis2, orthognalBasis1)

    objectProjectionOnOrthBasis1 = np.dot(objectRelPos, orthognalBasis1)
    if objectProjectionOnOrthBasis1 < 0:
        return False
    objectProjectionOnOrthBasis2 = np.dot(objectRelPos, orthognalBasis2)
    objectProjectionOnOrthBasis3 = np.dot(objectRelPos, orthognalBasis3)

    newCoordinate = [objectProjectionOnOrthBasis1, objectProjectionOnOrthBasis2, objectProjectionOnOrthBasis3]

    tt = [1, 0, 0]

    t = [objectProjectionOnOrthBasis1, objectProjectionOnOrthBasis2, 0]
    angle1 = np.arccos(np.dot(tt, t) / np.linalg.norm(t)) / 3.14159 * 180
    print angle1
    if abs(angle1) > 37:
        return False
    t = [objectProjectionOnOrthBasis1, 0, objectProjectionOnOrthBasis3]
    angle2 = np.arccos(np.dot(tt, t) / np.linalg.norm(t)) / 3.14159 * 180
    if abs(angle2) > 18.5:
        return False
    
    if objectProjectionOnOrthBasis3 < 0 and angle2 > 0:
        angle2 = -angle2

    if objectProjectionOnOrthBasis2 > 0 and angle1 > 0:
        angle1 = -angle1
    return angle1, angle2

def getState(angle, m, n):
    if not angle:
        print "The object can't be seen"
        return False
    angle1 = angle[0]
    angle2 = angle[1]
    unit1 = 2 * np.cos(angle1) / m
    unit2 = 2 * np.cos(angle2) / n
    state1 = np.cos(angle1) / unit1
    state2 = np.cos(angle2) / unit2
    if angle1 > 0:
        state1 = -state1
    if angle2 < 0:
        state2 = -state2
    return (state1, state2)

test = canSeeJudge(headForwardDirection(), objectRelPosition(name))
print test
print getState(test, 5,5)

46.3028718774
False
The object can't be seen
False


In [11]:
import math
print math.cos(75.0 / 180 * 3.14159)

0.25882011309


In [ ]:
def stateDetection(degree1, degree2, numStateX, numStateY):
    